In [1]:
# libraries 
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta

# https://ourworldindata.org/grapher/share-people-fully-vaccinated-covid
# https://ourworldindata.org/grapher/share-people-vaccinated-covid

# data directory
data_dir = "../../basins/"

# list of countries
countries = ['Sri Lanka', 'El Salvador', 'Morocco', 'Bolivia', 'Honduras',
             'Philippines', 'Indonesia', 'Pakistan', 'Rwanda', 'Bangladesh',
             'Kyrgyzstan', 'Egypt', 'Mozambique', 'Afghanistan', 'Angola',
             'Ghana', 'Zambia', "Côte d'Ivoire", 'Kenya', 'Uganda']


def get_doses(df_dose, country, column):
    if country == "Côte d'Ivoire":
        country_vacc = "Cote d'Ivoire"
    else:
        country_vacc = country

    N = pd.read_csv(os.path.join(data_dir, country, 'demographic/Nk_10.csv'))["value"].sum()

    df_country = df_dose.loc[df_dose.Entity == country_vacc]
    df_country.index = pd.to_datetime(df_country.Day)
    df_country = df_country.resample("D")[[column]].mean()
    #df_country.fillna(method='ffill', inplace=True)
    df_country.interpolate(method='linear', inplace=True)

    if column == "people_vaccinated_per_hundred":
        dose = "one"
    else:
        dose = "two"
        
    df_country[dose + "_dose_cumulative"] = df_country[column] / 100 * N
    df_country[dose + "_dose_daily"] = df_country[dose + "_dose_cumulative"].diff().fillna(method='bfill')
    
    return df_country


def get_doses_country(country):
    one_dose_country = get_doses(one_dose, country, "people_vaccinated_per_hundred")
    two_dose_country = get_doses(two_dose, country, "people_fully_vaccinated_per_hundred")
    dose_country = pd.merge(one_dose_country, two_dose_country, on='Day', how='outer')
    dose_country.fillna(value=0.0, inplace=True)
    if dose_country.shape[0] == 0:
        print('Country not found:', country)
    return dose_country


# import data
one_dose = pd.read_csv("./share-people-vaccinated-covid.csv")
two_dose = pd.read_csv("./share-people-fully-vaccinated-covid.csv")
two_dose.head()

,Entity,Code,Day,people_fully_vaccinated_per_hundred,145609-annotations
0,Afghanistan,AFG,2021-05-11,0.14,NaN
1,Afghanistan,AFG,2021-05-20,0.19,NaN
2,Afghanistan,AFG,2021-05-24,0.24,NaN
3,Afghanistan,AFG,2021-05-26,0.28,NaN
4,Afghanistan,AFG,2021-05-27,0.29,NaN


In [2]:
for country in countries:
    print(country)
    df = get_doses_country(country)
    df.to_csv(os.path.join(data_dir, country, 'vaccinations/vaccinations.csv'))

Sri Lanka
El Salvador
Morocco
Bolivia
Honduras
Philippines
Indonesia
Pakistan
Rwanda
Bangladesh
Kyrgyzstan
Egypt
Mozambique
Afghanistan
Angola
Ghana
Zambia
Côte d'Ivoire
Kenya
Uganda
